In [1]:
# https://adventofcode.com/2021/day/8
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
digits_default_str = {
     0: 'abcefg', 
     1: 'cf',
     2: 'acdeg',
     3: 'acdfg', 
     4: 'bcdf', 
     5: 'abdfg', 
     6: 'abdefg', 
     7: 'acf', 
     8: 'abcdefg', 
     9: 'abcdfg'
}

digits_default_len = {}
for k, v in digits_default_str.items():
    digits_default_len[k] = len(v)
    #print(f'{k}: {len(v)}')

len_unique = [1, 4, 7, 8]
len_5 = [2, 3, 5]
len_6 = [0, 6, 9]

In [3]:
def load_input(input_type=3):
    # sample input.
    if input_type==1:
        lines = [
            'acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf']
    
    # test input.
    elif input_type==2:
        lines = [
            'be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe', 
            'edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc', 
            'fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg', 
            'fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb', 
            'aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea', 
            'fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb', 
            'dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe', 
            'bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef', 
            'egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb', 
            'gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce']
    
    # real input.
    else:
        with open("day08.txt") as f:
            lines = f.read().strip().split('\n')
            
            
    x = [line.split('|')[0] for line in lines]
    combinations = [x_.split(' ')[:10] for x_ in x] 

    x = [line.split('|')[1] for line in lines]
    outputs = [x_.split(' ')[1:] for x_ in x]
    
    return combinations, outputs

In [4]:
def search_digit(combination, i):
    return [c for c in combination if len(c)==digits_default_len[i]]


def broken2default(digit_broken, translation_key):
    x = [translation_key.get(d, d) for d in digit_broken]
    x.sort()
    return ''.join(x)


def diff_str(str1, str2):
#    diff_ = [a for a in str1 if not a in str2]
    diff_1 = list(set(str1) - set(str2))
    diff_2 = list(set(str2) - set(str1))
    diff_  = diff_1 + diff_2  
    return diff_


def overlap_list(lst1, lst2):
    return list(set(lst1) & set(lst2))


def is_in_string(letters, string):
    return all([True if i in string else False for i in list(letters)])


def sort_letters(letters):
    letter_list = list(letters)
    letter_list.sort()
    return ''.join(letter_list)

In [5]:
## part 1.
combinations, outputs = load_input(input_type=2)
digits_lens = []
for digits in outputs:
    digits_len = [len(d) for d in digits]
    digits_lens.extend(digits_len)
#print(digits_lens)
total = 0
for i in [1, 4, 7, 8]:
    default_len = digits_default_len[i]
    total += digits_lens.count(default_len)
total

26

## part 2

In [6]:
def get_mapping(combination, explanation=False):
    # when the length is unique
    # the number is obvious.  
    digits_broken_str = {}
    for i in len_unique:
        digits_broken_str[i] = search_digit(combination, i)[0]

    # 'a' = 7('acf') - 1('cf')
    a = diff_str(digits_broken_str[7], digits_broken_str[1])
    a = a[0]
    if explanation:
        print(f"by comparing 1('cf') and 7('acf'), we know")
        print(f"[a] became {a[0]}")


    # [c, f] is known.
    cf = list(digits_broken_str[1])
    if explanation:
        print(f"[c, f] became {cf}")


    x = search_digit(combination, 2)
    digits_broken_str[3] = [
        x_ for x_ in x if is_in_string(digits_broken_str[7], x_)][0]
    if explanation:
        print("")
        print('among the strings with the length of 5')
        print("(2: 'acdeg', 3: 'acdfg', 5: 'abdfg')")
        print("the one which includes 7('acf') is 3('acdfg')")
        print(f"therefore, digits_broken_str[3] = '{digits_broken_str[3]}'")


    dg = diff_str(digits_broken_str[3], digits_broken_str[7])
    if explanation:
        print("")
        print("by subtracing 7('acf') from 3('acdfg')")
        print(f"[d, g] became {dg}")


    # 4: bcdf: cf + bd
    bd = diff_str(digits_broken_str[4], cf)
    if explanation:
        print("")
        print("by subtracing cf from 4('bcdf')")
        print(f"[b, d] became {bd}")


    # as dg is known...
    d = overlap_list(dg, bd)[0]
    g = diff_str(dg, d)[0]
    b = diff_str(bd, d)[0]
    if explanation:
        print("")
        print("by comparing [b, d] and [d, g]")
        print(f"[d] became {d} (included in both)")
        print(f"[b] became {b}")
        print(f"[g] became {g}")


    known_str = a + b + ''.join(cf) + d + g
    e = diff_str(digits_broken_str[8], known_str)[0]
    if explanation:
        print("")
        print("because e = 8('abcdefg') - a + b + cf + d + g")
        print(f"[e] became {e}")


    x = search_digit(combination, 2)
    # remove 3
    x.remove(digits_broken_str[3])
    y = diff_str(x[0], digits_broken_str[3])
    if e in y:
        digits_broken_str[2] = x[0]
        digits_broken_str[5] = x[1]
    else:
        digits_broken_str[2] = x[1]
        digits_broken_str[5] = x[0]
    y = diff_str(digits_broken_str[2], digits_broken_str[3])
    y.remove(e)
    f = y[0]

    y = diff_str(digits_broken_str[3], digits_broken_str[5])
    y.remove(b)
    c = y[0]
    if explanation:
        print("")
        print('among the strings with the length of 5')
        print("(2: 'acdeg', 3: 'acdfg', 5: 'abdfg')")
        print("difference between 2 and 3 is e and f")
        print("difference between 3 and 5 is c and b")
        print("given e and b are known...")
        print(f"[c] became {c}")
        print(f"[f] became {f}")


    ## set all the mappings.
    map_broken2default = {a: 'a'}
    map_broken2default[b] = 'b' 
    map_broken2default[c] = 'c'
    map_broken2default[d] = 'd' 
    map_broken2default[e] = 'e'
    map_broken2default[f] = 'f'
    map_broken2default[g] = 'g'

    return map_broken2default

In [17]:
def swap_key_value(dic):
    return dict([(value, key) for key, value in dic.items()])


def default2broken(x, map_default2broken):
    y = [map_default2broken.get(x_, x_) for x_ in list(x)]
    return ''.join(y)


def get_digits_broken_str(map_default2broken):
    digits_broken_str = {}
    for i, v in digits_default_str.items():
        digits_broken_str[i] = sort_letters(default2broken(v, map_default2broken))
    return digits_broken_str


def output2num(output):
    digits = []
    for digit_abc in output:
        digit_abc = sort_letters(digit_abc)
        digit = map_digits2num.get(digit_abc, digit_abc)
        digits.append(str(digit))
    return int(''.join(digits))

In [21]:
## part 2.
combinations, outputs = load_input(input_type=3)

nums = []
for combination, output in zip(combinations, outputs):
    map_broken2default = get_mapping(combination, explanation=False)

    map_default2broken = swap_key_value(map_broken2default)
    digits_broken_str = get_digits_broken_str(map_default2broken)

    map_digits2num = swap_key_value(digits_broken_str)

    nums.append(output2num(output))

np.sum(nums)

998900